In [2]:
import jieba
jieba.set_dictionary('ChineseDictionary/dict.txt.big')
import numpy as np
import pandas as pd
import operator
import matplotlib.pyplot as plt
%matplotlib inline
data = pd.read_excel("customer_record_data.xlsx")

In [3]:
data.head()

,類別,Unique ID,資料日期,客戶事件描述
0,Call Out,AA2765,20180102 141000,[ECP] 去電確認開戶文件 客戶說後來去買美元保單就不開戶了
1,Call Out,AA3910,20180102 144300,[ECP]不方說 IPO MAIL
2,Call Out,AA3167,20180102 154500,[ECP] IPO 忙 先MAIL
3,Call Out,AA2355,20180102 194359,家庭戶 媽媽說女兒不開了，因為打工收入不穩定
4,Call Out,AA2778,20180103 111132,推Q1


First, lets see how many unique IDs are there in the record

In [4]:
data = np.array(data)
unique_ids = set(data[:,1])
print("Amount of Unique IDs: "+str(len(unique_ids)))

Amount of Unique IDs: 572


Then we count times of the ID (we will call it customer) had called.

In [8]:
unique_id_called = dict()
for i in data:
    if i[1] in unique_id_called:
        unique_id_called[i[1]] += 1
    else:
        unique_id_called[i[1]] = 1

sorted_unique_id_called = sorted(unique_id_called.items(),key=operator.itemgetter(1))
# for i in range(1,len(unique_id_called)):
#     print(sorted_unique_id_called[-i])

draw_hist = dict()
for i in sorted_unique_id_called:
    if i[1] in draw_hist:
        draw_hist[i[1]] += 1
    else:
        draw_hist[i[1]] = 1
        
keys = np.array(list(draw_hist.keys()))
values = np.array(list(draw_hist.values( ) ) )
for i in range(len(keys)):
    print("called",keys[i],"time(s) for",values[i],"times")


called 1 time(s) for 350 times
called 2 time(s) for 123 times
called 3 time(s) for 50 times
called 4 time(s) for 20 times
called 5 time(s) for 15 times
called 6 time(s) for 8 times
called 7 time(s) for 4 times
called 8 time(s) for 1 times
called 14 time(s) for 1 times


Now, we look at some of the most common word in the message section.

In [9]:
import jieba.analyse
jieba.enable_parallel(8)
cnt = 0
for i in data:
    cnt = cnt + 1
    if cnt <= 15:
        print(jieba.lcut(i[3]))

Building prefix dict from /home/jupyter/DS/DataScience/HW1/ChineseDictionary/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u42f68d9ace05fa3d7ff1d69de828036b.cache
Loading model cost 1.936 seconds.
Prefix dict has been built succesfully.


['[', 'ECP', ']', ' ', '去電', '確認', '開戶', '文件', ' ', '客戶', '說', '後來', '去', '買', '美元', '保單', '就', '不', '開戶', '了']
['[', 'ECP', ']', '不方', '說', ' ', ' ', 'IPO', ' ', 'MAIL']
['[', 'ECP', ']', ' ', 'IPO', ' ', '忙', ' ', '先', 'MAIL']
['家庭', '戶', ' ', '媽媽', '說', '女兒', '不開', '了', '，', '因為', '打工', '收入', '不', '穩定']
['推', 'Q1']
['兒子', '要', '開戶', '未成年', ',', '不願辦', '網路', '要', '單筆', '買', '亞太', '複合', '債', '.']
['林', 'O', '屏', '216022', ' ', '家庭', '戶', ' ', '有', '可能', '有', '這戶', '下單', '又', '嫌', '麻煩']
['台幣', ' ', '瑞興', ' ', '轉介', '208']
['想', '買', '基金', ' ', '推', 'EC', ' ', '轉介', '之']
['某', '小姐', '要', '買', 'nnl', '新興', '市場', '債', ' ', '美元', '買', ' ']
['[', 'ECP', ']', '可能', '買回', ' ', ' ', '積', ' ', '單', ' ', '可以', '忍', '波動']
['Idris', '轉介']
['客戶', '來電', '說', '看到', '4', '年', 'IPO', ' ', '要', '美金', '的', ' ', '先寄', '開戶', '文件']
['去電', '確認', '外幣', '帳號', '是', '台北', '富邦']
['要', '用', '女兒', '名字', '開']


From the above we can see what is segmented. It is apparently some kind of junk. SO, instead, we search for keywords.

In [10]:
key_words = dict()
cnt = 0
for i in data:
    for j in jieba.analyse.extract_tags(i[3]):
        if j in key_words:
            key_words[j] += 1
        else:
            key_words[j] = 1
    cnt = cnt + 1
    if cnt <=15:
        print(jieba.analyse.extract_tags(i[3]))

key_words = sorted(key_words.items(),key=operator.itemgetter(1))

['開戶', '後來', '去電', 'ECP', '保單', '確認', '客戶', '文件', '美元']
['不方', 'ECP', 'IPO', 'MAIL']
['ECP', 'IPO', 'MAIL']
['媽媽', '穩定', '因為', '女兒', '不開', '打工', '家庭', '收入']
['Q1']
['複合', '網路', '單筆', '開戶', '不願辦', '亞太', '兒子', '未成年']
['216022', '麻煩', '下單', '這戶', '家庭', '可能']
['轉介', '台幣', '瑞興', '208']
['轉介', 'EC', '基金']
['市場', 'nnl', '新興', '小姐', '美元']
['ECP', '波動', '買回', '可能', '可以']
['轉介', 'Idris']
['先寄', 'IPO', '開戶', '來電', '客戶', '美金', '文件', '看到']
['富邦', '外幣', '去電', '帳號', '確認', '台北']
['女兒', '名字']


Now the above seems much more readable, although we may lose some of the information in the cutting stage. Some first thoughts upon seeing these datas.
1. It is mostly key word oriented and thus many verbs are removed
2. Given that it does mention some commodities, it is still hard to decide its intention.
    * For example, '覺得', '計價', '6%', '收益', '美金', '短期', '全球' we don't know whether the customer wants to long or short 美金.
3. There are unimportant verbs concerning selling commodities.
    * 解釋, '收到', '請客'
4. Abbreviations:
    * 國世銀 $\rightarrow$ 國泰世華銀行 ，富邦$\rightarrow$ 富邦銀行
    
The following blocks will show the most commonly appeared word in the service record.

In [26]:
for i in range(1,20):
    
    print(key_words[-i])

key_
    
len(key_words)

('ECP', 440)
('名單', 180)
('開戶', 163)
('Money', 148)
('客戶', 122)
('專刊', 92)
('退休', 84)
('電子報', 78)
('訂閱', 78)
('寄送', 76)
('追蹤', 75)
('分配', 65)
('IPO', 65)
('EC', 61)
('轉介', 59)
('寄回', 57)
('家庭', 54)
('收到', 47)
('表單', 46)


NameError: name 'key_' is not defined

The above method doesn't seems to be a good idea. Since, applying key word count to different customer is useless.
Lets look at what each customer is most interested in. Here we set a limit that a **regular** customer are those who at least called $3$ times.

In [13]:
regular_customer = []
regular_thresh = 1 # Change the number to define what the lower limit should be.
for i in unique_id_called:
    if unique_id_called[i] >= regular_thresh:
#         print("hit")
#         print(unique_id_called[i])
        regular_customer.append(i)
regular_customer = np.array(regular_customer)

df1 = pd.DataFrame([x for x in unique_id_called if unique_id_called[x]>= regular_thresh],columns=['Customer ID'])
df2 = pd.DataFrame([unique_id_called[x] for x in unique_id_called if unique_id_called[x]>= regular_thresh],columns=['Call time'])
df3 = pd.DataFrame(list("abc"),columns=['tags'])
merged_df = pd.concat([df1,df2,df3],axis=1)

cnt = 0
print_limt = 15 # update this to print more customers

for i in regular_customer:
    tag = list()
    # go through the data and find all those that matches "i"
    for j in data:
        if j[1] == i:
            # Change the method of cutting here.
            # I am using keyword search right now, which maybe somehow inaccurate considering the given data.
            tag.append(jieba.analyse.extract_tags( j[3]))
    
    
    tag = [item for sublist in tag for item in sublist]
    merged_df["tags"][cnt] = tag
    cnt += 1
merged_df.head(n=15)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Customer ID,Call time,tags
0,AA2861,2,"[人民幣, 姐姐, 先寄, 轉介, IPO, 期債, Q1, 銀行, vivian, 資料,..."
1,AA3024,1,"[台幣, 興趣, 對債, LINE, 基富通, 客戶, 只作, 好友, 不作, 境外]"
2,AA3476,1,"[現場, 富邦, 182313, EC, 兒子, 服務, 先買, 開戶, 一本萬利, 協助,..."
3,AA7870,2,"[ECP, 生日, 下亞高, 今天]"
4,AA2285,2,"[媽媽, 313128, 開戶, 呂張, 板信, 台幣, 寄給, 成年, 傢俱, 先不開, ..."
5,AA3533,2,"[爸爸, 寄給, 環球, 開戶, 處理, 一銀, 商銀, 20, RSP, 家庭, 擔心, ..."
6,AA2855,2,"[比較, 能源, NN, 台運, 外幣, 特別, Q1, 台幣, 興趣, 介紹, 28887..."
7,AA2868,1,"[勾錯, Cherry, 快遞, 請客, 稅務, ECP, 美國, 風險, 要採, 修改, 居民]"
8,AA7944,1,"[缺填, ECP, 掛退, email, 信箱]"
9,AA2885,1,"[期債, Q1, IPO, 客戶, 四年]"


The above shows how the merged dataframe. In the last column, it shows the tags "jieba" has cut off. Also, it is a merged version of all of the appeared tag.

The most reasonable way I think to conduct the EDA, would be to bring in some Hypothesis Testing techniques into this. However, most of those theory are applied to data concerning interval data or nominal data. That is, if we want a more theoretical explanatory about the data, it will be harder to prove.

Besides, we really lack in the amount of data. 1000 entries can not really show anything, especially there are about 500 unique ids.

In [27]:
# np.array(data[:,3])